In [49]:
import requests
import pandas as pd
import json
import re
import os
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
base_url = 'https://xecdapi.xe.com/v1'

In [54]:
def fetch_data(base_url, endpoint, params):
    auth = {
        'user': os.getenv('API_ID'),
        'password': os.getenv('API_KEY')
    }
    params.update(auth)
    r = requests.get(f'{base_url}/{endpoint}/', auth=(auth['user'], auth['password']), params=params)
    print(r)
    
    return r.json()

In [58]:
ops = {
    'from': 'USD',
    'to': ','.join(['NGN', 'GHS', 'KES', 'UGX', 'MAD', 'XOF', 'EGP']),
    'amount': 1,
    # 'obsolete': False,
    'inverse': True
}
convert_from = fetch_data(base_url=base_url, endpoint='convert_from', params=ops)

<Response [200]>


In [60]:
from pprint import pprint
pprint(convert_from)

{'amount': 1.0,
 'from': 'USD',
 'privacy': 'http://www.xe.com/privacy.php',
 'terms': 'http://www.xe.com/legal/dfs.php',
 'timestamp': '2022-08-20T00:00:00Z',
 'to': [{'inverse': 0.0961080919, 'mid': 10.4049511397, 'quotecurrency': 'MAD'},
        {'inverse': 0.1025671441, 'mid': 9.7497108719, 'quotecurrency': 'GHS'},
        {'inverse': 0.0023791748, 'mid': 420.313803586, 'quotecurrency': 'NGN'},
        {'inverse': 0.0002599058,
         'mid': 3847.5480026024,
         'quotecurrency': 'UGX'},
        {'inverse': 0.0083478581, 'mid': 119.791206724, 'quotecurrency': 'KES'},
        {'inverse': 0.0015307301,
         'mid': 653.2830422884,
         'quotecurrency': 'XOF'},
        {'inverse': 0.0522490318,
         'mid': 19.1391106462,
         'quotecurrency': 'EGP'}]}


In [77]:
def convert_to_dataframe(json_data):
    df = pd.DataFrame()
    quotecurrency = []
    mid = []
    inverse = []
    for idx in convert_from['to']:
        quotecurrency.append(idx['quotecurrency'])
        mid.append(idx['mid'])
        inverse.append(idx['inverse'])
    df['USD_to_currency_rate'] = mid
    df['currency_to_USD_rate'] = inverse
    df['currency_to'] = quotecurrency
    df['timestamp'] = convert_from['timestamp']
    df['currency_from'] = convert_from['from']
    columns = ['timestamp', 'currency_from', 'USD_to_currency_rate', 'currency_to_USD_rate', 'currency_to']
    df = df[columns]

    return df

result = convert_to_dataframe(convert_from)

In [82]:
from datetime import date
today = date.today()
today


datetime.date(2022, 8, 20)

In [84]:
result.to_csv(f'rates_{today}.csv', index=0)

In [85]:
f'rates_{today}.csv'

'rates_2022-08-20.csv'